In [6]:
import os
import pandas as pd

# Sets the path to the parent directory of RR classes
os.chdir("D:\\Study\\UniversityUW\\sem4\\RR\\RRcourse2024")

# Import data from the O*NET database, at ISCO-08 occupation level.
# The original data uses a version of SOC classification, but the data we load here
# are already cross-walked to ISCO-08 using: https://ibs.org.pl/en/resources/occupation-classifications-crosswalks-from-onet-soc-to-isco/

# The O*NET database contains information for occupations in the USA, including
# the tasks and activities typically associated with a specific occupation.

task_data = pd.read_csv("Data\\onet_tasks.csv")
# isco08 variable is for occupation codes
# the t_* variables are specific tasks conducted on the job

# read employment data from Eurostat
# These datasets include quarterly information on the number of workers in specific
# 1-digit ISCO occupation categories. (Check here for details: https://www.ilo.org/public/english/bureau/stat/isco/isco08/)

isco_data = pd.read_excel("Data\\Eurostat_employment_isco.xlsx", sheet_name=None)

# Combine data for each ISCO category into a single DataFrame
isco_combined = pd.concat([df.assign(occupation=key) for key, df in isco_data.items()])

# Focus on three countries, allowing for easier adaptation to run for all countries
countries = ["Belgium", "Spain", "Poland"]

# Calculate worker totals for each country
total_workers = {country: isco_combined[country].sum() for country in countries}

# Display total workers for each country
total_workers


{'Belgium': 365116.3, 'Spain': 1453426.5, 'Poland': 1254584.7}

In [7]:
import pandas as pd

# Let's merge all these datasets. We'll need a column that stores the occupation categories
for i, df in enumerate(isco_data.values(), start=1):
    df['ISCO'] = i

# Combine all ISCO category DataFrames into a single DataFrame with an additional 'occupation' column
all_data = pd.concat(isco_data.values(), ignore_index=True)

# Calculate total workers for each country and merge into the combined DataFrame
for country in countries:
    total_country = total_workers[country]
    all_data[f"total_{country}"] = total_country

# Calculate the share of each occupation among all workers in a period-country
for country in countries:
    all_data[f"share_{country}"] = all_data[country] / all_data[f"total_{country}"]

all_data.head()  # Display the first few rows of the combined DataFrame for verification


,TIME,European Union - 28 countries (2013-2020),Belgium,Czechia,Denmark,Spain,Italy,Lithuania,Poland,Finland,Sweden,ISCO,total_Belgium,total_Spain,total_Poland,share_Belgium,share_Spain,share_Poland
0,2011-Q1,210428.6,4418.6,4758.8,2570.1,18277.7,22156.6,1206.9,15126.8,2366.7,4406.2,1,365116.3,1453426.5,1254584.7,0.012102,0.012576,0.012057
1,2011-Q2,212606.7,4508.3,4800.2,2605.0,18475.7,22335.5,1227.8,15331.7,2468.5,4519.4,1,365116.3,1453426.5,1254584.7,0.012348,0.012712,0.012221
2,2011-Q3,213154.9,4452.9,4819.7,2606.6,18337.8,22196.1,1233.7,15440.9,2468.6,4578.9,1,365116.3,1453426.5,1254584.7,0.012196,0.012617,0.012308
3,2011-Q4,211867.3,4502.1,4806.9,2568.8,17992.4,22171.5,1234.5,15351.7,2410.0,4487.9,1,365116.3,1453426.5,1254584.7,0.012331,0.012379,0.012236
4,2012-Q1,209511.3,4451.4,4759.6,2549.1,17615.5,22030.3,1222.4,15149.6,2383.4,4428.3,1,365116.3,1453426.5,1254584.7,0.012192,0.012120,0.012075


In [8]:
# Now let's look at the task data. We want the first digit of the ISCO variable only
import pandas as pd
import numpy as np
import re

task_data["isco08_1dig"] = task_data["isco08"].astype(str).str[:1].astype(int)

# And we'll calculate the mean task values at a 1-digit level
aggdata = task_data.groupby(["isco08_1dig"]).mean()
aggdata = aggdata.drop(columns=["isco08"])

# We'll be interested in tracking the intensity of Non-routine cognitive analytical tasks
# Using a framework reminiscent of the work by David Autor.

# These are the ones we're interested in:
# Non-routine cognitive analytical
# 4.A.2.a.4 Analyzing Data or Information
# 4.A.2.b.2 Thinking Creatively
# 4.A.4.a.1 Interpreting the Meaning of Information for Others

# Let's combine the data.
combined = pd.merge(all_data, aggdata, left_on='ISCO', right_on='isco08_1dig', how='left')

# Standardisation -> getting the mean to 0 and std. dev. to 1.
# Let's do this for each of the variables that interests us:

# List of task variables of interest
tasks_of_interest = [
    "t_4A2a4",  # Analyzing Data or Information
    "t_4A2b2",  # Thinking Creatively
    "t_4A4a1"   # Interpreting the Meaning of Information for Others
]

for task in tasks_of_interest:
    for country in countries:
        mean_task = np.average(combined[task], weights=combined[f"share_{country}"])
        std_task = np.sqrt(np.average((combined[task] - mean_task)**2, weights=combined[f"share_{country}"]))
        combined[f"std_{country}_{task}"] = (combined[task] - mean_task) / std_task

combined.head()  # Display the first few rows of the combined DataFrame for verification


,TIME,European Union - 28 countries (2013-2020),Belgium,Czechia,Denmark,Spain,Italy,Lithuania,Poland,Finland,...,t_4A3b5_rev,std_Belgium_t_4A2a4,std_Spain_t_4A2a4,std_Poland_t_4A2a4,std_Belgium_t_4A2b2,std_Spain_t_4A2b2,std_Poland_t_4A2b2,std_Belgium_t_4A4a1,std_Spain_t_4A4a1,std_Poland_t_4A4a1
0,2011-Q1,210428.6,4418.6,4758.8,2570.1,18277.7,22156.6,1206.9,15126.8,2366.7,...,4.270759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-Q2,212606.7,4508.3,4800.2,2605.0,18475.7,22335.5,1227.8,15331.7,2468.5,...,4.270759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-Q3,213154.9,4452.9,4819.7,2606.6,18337.8,22196.1,1233.7,15440.9,2468.6,...,4.270759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-Q4,211867.3,4502.1,4806.9,2568.8,17992.4,22171.5,1234.5,15351.7,2410.0,...,4.270759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-Q1,209511.3,4451.4,4759.6,2549.1,17615.5,22030.3,1222.4,15149.6,2383.4,...,4.270759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
